**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML,display
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act executes the action learned on the environment. It enables to do exploration/exploitation using an $\epsilon$-greedy policy : we act randomly with a probability $\epsilon$. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
#         self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3 # Probability of success when creating bonus and malus
epochs_train = 50 # set small when debugging
epochs_test = 30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video width="320" height="240" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array $\textit position$ is the map on which we are playing, used to define the borders.

The array $\textit board$ contains the reward of each cell on the map.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(low = 0,high = 4,size = 1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
            action = agent.act(state)
            
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward >0:
                win += reward
            if reward <0:
                lose -= reward
        
        # Save as a mp4
        if e%10:
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random9.mp4'))

Win/lose count 7.5/6.0. Average score (1.5)
Win/lose count 7.5/17.0. Average score (-4.0)
Win/lose count 5.5/23.0. Average score (-8.5)
Win/lose count 12.0/8.0. Average score (-5.375)
Win/lose count 6.0/7.0. Average score (-4.5)
Win/lose count 13.0/14.0. Average score (-3.9166666666666665)
Win/lose count 8.0/11.0. Average score (-3.7857142857142856)
Win/lose count 12.0/15.0. Average score (-3.6875)
Win/lose count 9.5/7.0. Average score (-3.0)
Win/lose count 9.0/8.0. Average score (-2.6)
Win/lose count 14.5/14.0. Average score (-2.3181818181818183)
Win/lose count 6.5/16.0. Average score (-2.9166666666666665)
Win/lose count 16.0/9.0. Average score (-2.1538461538461537)
Win/lose count 11.0/14.0. Average score (-2.2142857142857144)
Win/lose count 10.5/17.0. Average score (-2.5)
Win/lose count 12.0/12.0. Average score (-2.34375)
Win/lose count 9.5/15.0. Average score (-2.5294117647058822)
Win/lose count 10.0/18.0. Average score (-2.8333333333333335)
Win/lose count 10.0/14.0. Average score (

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Question 1** We know that

$$ Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s_{0},a_{0}) + \sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s,a) + \gamma \sum_{1 \leq t}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$


$$ Q^\pi(s,a)=E_{(s',a')\sim p(.|s,a)}[ r(s,a) + E_{p^{\pi}}[ \gamma \sum_{t}\gamma^{t}r(s_{t}^\prime,a_{t}^\prime)|s_{0}^\prime=s^\prime,a_{0}^\prime=a^\prime] ] \> $$

where $ (s_{t},a_{t}) = (s_{t}^\prime,a_{t}^\prime) $ and $ (s^\prime,a^\prime) = (s_1 , a_1)$  )

In the last equation, we recognize the Q-function as the second member.

$$ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$


**Question 2**

We saw in class that:

$$ \pi^*(s) = \max_a Q^*(s,a) $$

which implies, using the first question that : 

$$ Q^{*}(s,a)=E_{(s',a')\sim \pi^* (.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',\pi^*(s'))] $$

and eventually, we get :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

**Question 3**

Using this cost function : 
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

will enable us to satisfy the Bellman equation. 

In fact we saw in the question 2 that the Q-value function is optimal if and only if the difference is null.
It is therefore a plausible objective.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        self.memory = self.memory[-self.max_memory:]

    def random_access(self):
        i = np.random.randint(0,len(self.memory))
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.layers import Flatten,Dense,Conv2D

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        chosen_action = np.argmax(self.model.predict(s.reshape((1,s.shape[0],s.shape[1],s.shape[2])))[0]) #Output of shape (1,4)
        return chosen_action

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_ #shape (5,5,2)
            target_q[i] = self.model.predict(s_.reshape((1,s_.shape[0],s_.shape[1],s_.shape[2])))[0] #Prediction of the old network
            
            if game_over_:
                target_q[i,a_] = r_
            else:
                target_q[i,a_] = r_ + self.discount*max(self.model.predict(n_s_.reshape((1,n_s_.shape[0],n_s_.shape[1],n_s_.shape[2])))[0])

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten(input_shape = (5,5,self.n_state)))
        model.add(Dense(32,activation = 'relu'))
        
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0063 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/050 | Loss 0.0195 | Win/lose count 4.5/2.0 (2.5)
Epoch 002/050 | Loss 0.0173 | Win/lose count 2.0/5.0 (-3.0)
Epoch 003/050 | Loss 0.0162 | Win/lose count 3.0/3.0 (0.0)
Epoch 004/050 | Loss 0.0073 | Win/lose count 3.0/1.0 (2.0)
Epoch 005/050 | Loss 0.0101 | Win/lose count 4.5/2.0 (2.5)
Epoch 006/050 | Loss 0.0076 | Win/lose count 3.5/1.0 (2.5)
Epoch 007/050 | Loss 0.0497 | Win/lose count 0.5/2.0 (-1.5)
Epoch 008/050 | Loss 0.0058 | Win/lose count 2.5/2.0 (0.5)
Epoch 009/050 | Loss 0.0118 | Win/lose count 1.5/5.0 (-3.5)
Epoch 010/050 | Loss 0.0438 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/050 | Loss 0.0126 | Win/lose count 5.5/4.0 (1.5)
Epoch 012/050 | Loss 0.0100 | Win/lose count 7.5/7.0 (0.5)
Epoch 013/050 | Loss 0.0083 | Win/lose count 6.0/3.0 (3.0)
Epoch 014/050 | Loss 0.0065 | Win/lose count 4.0/8.0 (-4.0)
Epoch 015/050 | Loss 0.0644 | Win/lose count 3.5/0 (3.5)
Epoch 016/050 | Loss 0.0654 | Win/lose count 5.5/2.0

In [14]:
HTML(display_videos('fc_train50.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64,(2,2),input_shape = (5,5,self.n_state),activation = 'relu'))
        model.add(Conv2D(32,(2,2),activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        print(model.summary())
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 4, 4, 64)          576       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          8224      
_________________________________________________________________
flatten_2 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 1156      
Total params: 9,956
Trainable params: 9,956
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/050 | Loss 0.0056 | Win/lose count 3.5/5.0 (-1.5)
Epoch 001/050 | Loss 0.0134 | Win/lose count 2.0/4.0 (-2.0)
Epoch 002/050 | Loss 0.0044 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/050 | Loss 0.0116 | Win/lose count 9.0/3.0 (6.0)
Epoch 004/050 | Loss 0.0057 |

In [17]:
HTML(display_videos('cnn_train50.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [18]:
epochs_test = 30

env = Environment(grid_size=size, max_time=T,temperature=0.7)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 4, 64)          576       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 32)          8224      
_________________________________________________________________
flatten_3 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 1156      
Total params: 9,956
Trainable params: 9,956
Non-trainable params: 0
_________________________________________________________________
None
Test of the CNN
Win/lose count 6.5/3.0. Average score (3.5)
Win/lose count 19.0/2.0. Average score (10.25)
Win/lose count 10.5/0. Average score (10.333333333333334)
Win/lose count 22.5/4.0. Average score (12.375)
Win/lose count 18.5/5.0. Average score (12.6)
Win/lose

In [28]:
HTML(display_videos('fc_test14.mp4'))

In [27]:
HTML(display_videos('cnn_test14.mp4'))

We notice that the network using convolutional layers performs better than the one using fully connected layers. We can see that as the temperature increases, the score increases which is intuitive due to the presence of more rewards in the map.

Also we notice that there is a lack of exploration. It goes in a direction but comes back in the same direction.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [32]:
def train_explore(agent,env,epoch,decay,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    epsilon_init = agent.epsilon
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        agent.epsilon = epsilon_init
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            agent.set_epsilon(agent.epsilon*decay)
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.malus_position = np.zeros((grid_size,grid_size))
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        
        if train:
            reward = -self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [33]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.7)
agent = DQN_CNN(size, lr=.1, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train,decay=0.8, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 4, 4, 64)          832       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 32)          8224      
_________________________________________________________________
flatten_6 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 1156      
Total params: 10,212
Trainable params: 10,212
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/050 | Loss 0.0048 | Win/lose count 8.0/21.10000000000004 (-13.10000000000004)
Epoch 001/050 | Loss 0.0082 | Win/lose count 10.100000000000003/21.300000000000036 (-11.200000000000033)
Epoch 002/050 | Loss 0.0038 | Win/lose count 9.800000000000002/20.900000

In [42]:
HTML(display_videos('cnn_train_explore40.mp4'))

In [44]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore28.mp4'))

Win/lose count 24.799999999999976/13.899999999999967. Average score (10.90000000000001)
Win/lose count 10.800000000000004/17.399999999999977. Average score (2.150000000000018)
Win/lose count 16.400000000000006/16.99999999999997. Average score (1.2333333333333567)
Win/lose count 18.0/16.59999999999997. Average score (1.2750000000000252)
Win/lose count 30.799999999999955/13.399999999999968. Average score (4.500000000000017)
Win/lose count 39.199999999999925/11.299999999999976. Average score (8.400000000000006)
Win/lose count 21.999999999999986/15.59999999999996. Average score (8.114285714285723)
Win/lose count 39.99999999999992/11.099999999999978. Average score (10.7125)
Win/lose count 31.59999999999995/12.199999999999973. Average score (11.677777777777775)
Win/lose count 33.199999999999946/11.799999999999974. Average score (12.649999999999995)
Win/lose count 30.799999999999955/15.399999999999961. Average score (12.899999999999995)
Win/lose count 43.19999999999991/12.29999999999998. Aver

In [48]:
HTML(display_videos('cnn_test_explore21.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***